# Fantasy Football - Choosing the Best Team
**Logan Colla (lcolla@wisc.edu), Nate Pritzl (npritzl@wisc.edu)**

### Table of Contents
1. [Introduction](#1.-Introduction)
2. [Mathematical Model](#2.-Mathematical-model)
3. [Solution](#3.-Solution)
4. [Results and Discussion](#4.-Results-and-discussion)
5. [Conclusion](#5.-Conclusion)

# 1. Introduction

Gambling and fantasy sports has grown tremendously in popularity.  Our goal is to create an optimizer that increases the chances of winning.  Traditionally, this is done using player projections for the objective function, and constraints on salary and position.  However, player projections isn't the only thing that should be used in the objective.  Knowing what players your opponents picked can change the optimal roster and increase your chances of winning.  Player projections and owner percentage are unknowns, and therefore should be determined using regression.  We will use the least squares technique to determine these quantities if we have time.  The main focus of our project is determine the optimized roster given the player projections and owner percentage. Data for player projections and positions can be found on espn.com.  Player ownership percentage can be found on dfsreport.com.  Salary can be found on fandual.com.

# 2. Mathematical Model

In [19]:
using LaTeXStrings 

\[x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}\]

LoadError: LoadError: syntax: "\" is not a unary operator
while loading In[19], in expression starting on line 3

# 3. Solution

In [25]:
using DataFrames
raw = DataFrame(readtable("csprojectdata.csv", header = true))

week = 12

if week==3
    week = week+1
elseif week == 12 || week == 13
    week = week-1
elseif week == 15
    week = week-2
elseif week == 5 || week == 6 || week == 7 || week == 8 || week == 9 || week == 10
    week = week
else
    println("Invalid Week")
end
df = raw[:,[1:3;week;week+10;week+20]]

,Player,Position,Team,Proj_12,Own_12,Cost_12
1,Tom Brady,QB,NE,17.02,0.036,8000
2,Cam Newton,QB,CAR,18.64,0.015,7100
3,Ben Roethlisberger,QB,PIT,17.24,0.015,6700
4,Nick Foles,QB,LA,9.76,0.015,5000
5,Andrew Luck,QB,IND,0.0,0.132,2500
6,Marcus Mariota,QB,TEN,17.12,0.015,5500
7,Andy Dalton,QB,CIN,17.52,0.015,6100
8,Ryan Fitzpatrick,QB,NYJ,17.26,0.015,5000
9,Blake Bortles,QB,JAX,18.52,0.015,5900
10,Derek Carr,QB,OAK,17.48,0.015,6300


In [24]:
using JuMP

N = size(raw)[1]
salaryCap = 35000
QBlimit = 1
RBlimit = 2
WRlimit = 2
TElimit = 1

m = Model()
@defVar(m, p[1:N], Bin)
@addConstraint(m, sum{p[i]*df[i,6], i=1:N} <= salaryCap)
@addConstraint(m, sum{p[i], i = 1:23} == QBlimit)
@addConstraint(m, sum{p[i], i = 24:63} == RBlimit)
@addConstraint(m, sum{p[i], i = 64:86} == TElimit)
@addConstraint(m, sum{p[i], i = 87:127} == WRlimit)
@setObjective(m, Max, sum{p[i]*df[i,4], i=1:N})
solve(m)

xopt=getValue(p)
for i = 1:N
    if xopt[i]==1
        println(df[i,:Player])
    end
end

Blake Bortles
Doug Martin
Darren McFadden
Jason Witten
DeAndre Hopkins
Nate Washington


# 4. Results and Discussion

# 5. Conclusion